In [4]:
import pandas as pd

label_mapping = {
    'negative': 0,
    'positive': 1,
    'neutral': 2
}
inverse_label_mapping = {
    0: 'negative',
    1: 'positive',
    2: 'neutral'
}

pred = pd.read_csv('DS/Twitter-US-Airline-Sentiment/analysis_selfRef/noPre/deberta-pred.csv')
label_counts = pred['Original_Label'].value_counts()
print("\033[92mTotal:", pred.shape[0], "\033[0m")
print("\033[92mOriginal labels count:\033[0m")
print("\033[92m", label_counts, "\033[0m")

def test(model_name, pre_processing):
    print("_____________________________________________________________________________")
    pre = "dataPre" if pre_processing else "noPre"
    print(f"Pre-trained model: {model_name}")

    original_path = f'DS/Twitter-US-Airline-Sentiment/analysis_selfRef/{pre}/{model_name}-pred.csv'
    rewrite_path = f'DS/Twitter-US-Airline-Sentiment/analysis_selfRef/{pre}/{model_name}-rewrite-pred.csv'
    sample_save_path = f'DS/Twitter-US-Airline-Sentiment/analysis_selfRef/{pre}/sample-{model_name}.csv'
    
    pred1 = pd.read_csv(original_path)
    pred2 = pd.read_csv(rewrite_path)
    pred1['pred_label_rewrite'] = pred2['pred_label']
    merged = pred1
    #merged.to_csv('merged_output.csv', index=False, encoding='utf-8')

    pred1_correct = merged['pred_label'] == merged['Original_Label'].map(label_mapping)
    pred2_correct = merged['pred_label_rewrite'] == merged['Original_Label'].map(label_mapping)

    pred1_correct_and_pred2_correct = (pred1_correct & pred2_correct).sum()
    pred1_incorrect_and_pred2_correct = ((~pred1_correct) & pred2_correct).sum()
    pred1_incorrect_and_pred2_incorrect = ((~pred1_correct) & (~pred2_correct)).sum()
    pred1_correct_and_pred2_incorrect = (pred1_correct & (~pred2_correct)).sum()

    print(f"\033[93mI: {pred1_correct_and_pred2_correct}, II: {pred1_incorrect_and_pred2_correct}, III: {pred1_incorrect_and_pred2_incorrect}, IV: {pred1_correct_and_pred2_incorrect}\033[0m")

    # 每种情况的数量
    count_pred1_correct_and_pred2_correct = (merged[pred1_correct & pred2_correct]['Original_Label'].value_counts())
    count_pred1_incorrect_and_pred2_correct = (merged[~pred1_correct & pred2_correct]['Original_Label'].value_counts())
    count_pred1_incorrect_and_pred2_incorrect = (merged[~pred1_correct & ~pred2_correct]['Original_Label'].value_counts())
    count_pred1_correct_and_pred2_incorrect = (merged[pred1_correct & ~pred2_correct]['Original_Label'].value_counts())

    # 打印结果
    print("Pred1 Correct and Pred2 Correct:")
    print(count_pred1_correct_and_pred2_correct)
    print("\nPred1 Incorrect and Pred2 Correct:")
    print(count_pred1_incorrect_and_pred2_correct)
    print("\nPred1 Incorrect and Pred2 Incorrect:")
    print(count_pred1_incorrect_and_pred2_incorrect)
    print("\nPred1 Correct and Pred2 Incorrect:")
    print(count_pred1_correct_and_pred2_incorrect)

    samples = {
        'I': merged[pred1_correct & pred2_correct],
        'II': merged[~pred1_correct & pred2_correct],
        'III': merged[~pred1_correct & ~pred2_correct],
        'IV': merged[pred1_correct & ~pred2_correct]
    }

    results = []

    # 临时关闭 SettingWithCopyWarning
    pd.options.mode.chained_assignment = None
    for condition, data in samples.items():
        if data.shape[0] > 0:
            data['pred_label'] = data['pred_label'].map(inverse_label_mapping)
            data['pred_label_rewrite'] = data['pred_label_rewrite'].map(inverse_label_mapping)
            for index, row in data.iterrows():
                results.append({
                    'Condition': condition,
                    'Correct_label': row['Original_Label'],
                    'Text': row['Text'],
                    'Pred': row['pred_label'],
                    'Text-Rewrite': row['Text-Rewrite'],
                    'Pred-Rewrite': row['pred_label_rewrite']
                })
    pd.options.mode.chained_assignment = 'warn'
                
    results_df = pd.DataFrame(results)
    results_df.to_csv(sample_save_path, index=False, encoding='utf-8')

test("bert", False)
test("roberta", False)
test("albert", False)
test("deberta", False)
test("electra", False)

print("\n")
#test("bert", True)
#test("roberta", True)
#test("albert", True)
#test("deberta", True)
#test("Electra, True)

Total: 2928 
Original labels count:
 Original_Label
negative    1889
neutral      580
positive     459
Name: count, dtype: int64 
_____________________________________________________________________________
Pre-trained model: bert
I: 2348, II: 158, III: 287, IV: 135
Pred1 Correct and Pred2 Correct:
Original_Label
negative    1676
positive     343
neutral      329
Name: count, dtype: int64

Pred1 Incorrect and Pred2 Correct:
Original_Label
neutral     66
negative    55
positive    37
Name: count, dtype: int64

Pred1 Incorrect and Pred2 Incorrect:
Original_Label
neutral     142
negative     92
positive     53
Name: count, dtype: int64

Pred1 Correct and Pred2 Incorrect:
Original_Label
negative    66
neutral     43
positive    26
Name: count, dtype: int64
_____________________________________________________________________________
Pre-trained model: roberta
I: 2405, II: 131, III: 274, IV: 118
Pred1 Correct and Pred2 Correct:
Original_Label
negative    1731
positive     345
neutral      